学习点：

1.邮箱的授权  
2.schedule：在证券交易中的制度中，有交易和休市时间，要实现程序的定时运行，该库必不可少，详见程序部分对该库用法的介绍。  
3.smtplib：该库主要实现电子邮件的发送。  
4.sys：在交易日的15:00以后已经闭市，为避免资源的浪费，此时可以调用sys.exit()方法实现程序的自动退出。  
5.pyinstaller：用该库可以将程序打包成可执行的exe格式文件，便于程序的运行  

In [14]:
!pip install tushare
!pip install smtplib

ERROR: Could not find a version that satisfies the requirement smtplib (from versions: none)
ERROR: No matching distribution found for smtplib


In [20]:
import datetime
import tushare as ts
import smtplib
from email.mime.text import MIMEText

In [4]:
def pd_ztjytime():#判断是否是交易时间
    now_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')         # 将datetime对象转换为格式化字符串
    now_datetime = datetime.datetime.strptime(now_time, '%Y-%m-%d %H:%M:%S') # 将格式化时间字符串转换回datetime对象
    d1 = datetime.datetime.strptime(datetime.datetime.now().strftime('%Y-%m-%d') + ' 11:30:01', '%Y-%m-%d %H:%M:%S')
    d2 = datetime.datetime.strptime(datetime.datetime.now().strftime('%Y-%m-%d') + ' 13:00:00', '%Y-%m-%d %H:%M:%S')
    delta1 = (now_datetime - d1).total_seconds()
    delta2 = (d2-now_datetime).total_seconds()
    if delta1>0 and delta2>0 : #在暂停交易的时间内
        return True  #在暂停的交易时间范围内，返回 True
    else:
        return False #不在暂停的交易时间范围内，返回 False

In [11]:
def get_now_jiage(code):
   df = ts.get_realtime_quotes(code)[['name','price','pre_close','date','time']]
   return df

In [21]:
def send_Email(Email_address, email_text):
    from_addr = '2330278295@qq.com' #发出电子邮件的地址
    password = '************'   #发出电子邮件的密码
    title = '股票价格异动监控消息-' + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')  #电子邮件的标题
    msg = MIMEText(email_text, 'html', 'utf-8') #电子邮件的格式是HTML
    msg['From'] = from_addr
    msg['To'] = Email_address
    msg['Subject'] = title

    try:
        server = smtplib.SMTP_SSL('smtp.qq.com', 465)
        server.login(from_addr, password)  # 发送邮件
        server.send_message(msg)
        server.quit()

        # print(Email_address+'  send success!')
        #send_info.append(Email_address + '  send success!\n')
    except Exception as e:
        a+1
        # print(e)
        #send_info.append(e + '\n')
        #send_info.append(Email_address + ' send failed!\n')
        # print(Email_address+' send failed!')

In [13]:
def do_programe(code):
    if pd_ztjytime()==False: #判断是否在暂停交易的时间范围内
        info=get_now_jiage(code) #调用方法获取当前的DataFrame
        now_jiage=float(info['price'][0]) #获取现价
        name=info['name'][0] #获取证券名称
        pre_close=float(info['pre_close'][0]) #获取昨日收盘价
        riqi=info['date'][0] #获取现价对应的日期
        sj=info['time'][0] #获取价格对应的时间
        now_zdie=round((now_jiage-pre_close)/pre_close*100,2) #计算现在的涨跌幅
        all_zdie=round((now_jiage-cbj)/cbj*100,2)  #计算股票持有期间内总的涨跌幅，其中cbj为购买时候的成本价，需要约定全局变量
        now_shizhi=round(shuliang*now_jiage,2) #计算股票现在的市值，其中shuliang为购买股票的数量，需要约定为全局变量
        ykui=round(now_shizhi-cbj*shuliang,2)  #计算股票现在总的盈亏
        if (abs(now_zdie)>=3 and abs(now_zdie)<3.09) or (abs(now_zdie)>=6  and abs(now_zdie)<6.05)  or (abs(now_zdie)>=9 and  abs(now_zdie)<9.1) : #判断现在的涨跌幅是否在目标范围内
            email_comment = []
            email_comment.append('<html>')
            email_comment.append('<b><p><h3><font size="2" color="black">您好：</font></h4></p></b>')
            email_comment.append('<p><font size="2" color="#000000">根据设置参数，现将监控到'+name+'('+str(code)+')的证券价格异动消息汇报如下：</font></p>')
            email_comment.append('<table border="1px" cellspacing="0px"   width="600" bgcolor=' + color_bg_fg + ' style="border-collapse:collapse">')

            email_comment.append('<tr>')
            email_comment.append('<td align="center"><b>序号</b></td>')
            email_comment.append('<td align="center"><b>购买单价</b></td>')
            email_comment.append('<td align="center"><b>持股数</b></td>')
            email_comment.append('<td align="center"><b>现价</b></td>')
            email_comment.append('<td align="center"><b>现涨跌幅</b></td>')
            email_comment.append('<td align="center"><b>总涨跌幅</b></td>')
            email_comment.append('<td align="center"><b>现市值</b></td>')
            email_comment.append('<td align="center"><b>盈亏额</b></td>')
            email_comment.append('<td align="center"><b>异动时间</b></td>')
            email_comment.append('</tr>')

            email_comment.append('<tr>')
            email_comment.append('<td align="center">'+str(1)+'</td>')
            email_comment.append('<td align="center">'+str(cbj) + '</td>')
            email_comment.append('<td align="center">' + str(shuliang) + '</td>')
            email_comment.append('<td align="center">' + str(now_jiage) +'</td>')
            email_comment.append('<td align="center">' + str(now_zdie) + '%</td>')
            email_comment.append('<td align="center">' + str(all_zdie) + '%</td>')
            email_comment.append('<td align="center">' + str(now_shizhi) + '元</td>')
            email_comment.append('<td align="center">' + str(ykui) + '元</td>')
            email_comment.append('<td align="center">' + str(riqi) +' '+str(sj) +'</td>')
            email_comment.append('</tr>')
            email_comment.append('</table>')
            email_comment.append('<p><font size="2" color="black">祝：股市天天红，日日发大财！</font></p>')
            email_comment.append('</html>')
            send_msg = '\n'.join(email_comment)
            send_Email(email_add[0], send_msg)

,name,price,pre_close,date,time
0,贵州茅台,1728.000,1726.000,2023-10-18,15:00:01


In [ ]:
nquhnrncaorkdjdf

In [ ]:
def run():
    while True:
        do_programe('600905')
        now_time=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        d1 = datetime.datetime.strptime(now_time, '%Y-%m-%d %H:%M:%S')
        d2 = datetime.datetime.strptime(datetime.datetime.now().strftime('%Y-%m-%d')+' 15:00:00', '%Y-%m-%d %H:%M:%S')
        delta = d2 - d1
        if delta.total_seconds()<=0:
          sys.exit()
        time.sleep(1)

In [ ]:
if __name__ == '__main__':
    schedule.every().day.at("09:30").do(run)
    while True:
        schedule.run_pending()
        time.sleep(1)

In [22]:
code = '600905'
cbj = 1
shuliang = 100
color_bg_fg = '#0000FF'
email_add = ['**********']
if pd_ztjytime()==False: #判断是否在暂停交易的时间范围内
        info=get_now_jiage(code) #调用方法获取当前的DataFrame
        now_jiage=float(info['price'][0]) #获取现价
        name=info['name'][0] #获取证券名称
        pre_close=float(info['pre_close'][0]) #获取昨日收盘价
        riqi=info['date'][0] #获取现价对应的日期
        sj=info['time'][0] #获取价格对应的时间
        now_zdie=round((now_jiage-pre_close)/pre_close*100,2) #计算现在的涨跌幅
        all_zdie=round((now_jiage-cbj)/cbj*100,2)  #计算股票持有期间内总的涨跌幅，其中cbj为购买时候的成本价，需要约定全局变量
        now_shizhi=round(shuliang*now_jiage,2) #计算股票现在的市值，其中shuliang为购买股票的数量，需要约定为全局变量
        ykui=round(now_shizhi-cbj*shuliang,2)  #计算股票现在总的盈亏
        if (abs(now_zdie)>=0 and abs(now_zdie)<3.09) or (abs(now_zdie)>=6  and abs(now_zdie)<6.05)  or (abs(now_zdie)>=9 and  abs(now_zdie)<9.1) : #判断现在的涨跌幅是否在目标范围内
            email_comment = []
            email_comment.append('<html>')
            email_comment.append('<b><p><h3><font size="2" color="black">您好：</font></h4></p></b>')
            email_comment.append('<p><font size="2" color="#000000">根据设置参数，现将监控到'+name+'('+str(code)+')的证券价格异动消息汇报如下：</font></p>')
            email_comment.append('<table border="1px" cellspacing="0px"   width="600" bgcolor=' + color_bg_fg + ' style="border-collapse:collapse">')

            email_comment.append('<tr>')
            email_comment.append('<td align="center"><b>序号</b></td>')
            email_comment.append('<td align="center"><b>购买单价</b></td>')
            email_comment.append('<td align="center"><b>持股数</b></td>')
            email_comment.append('<td align="center"><b>现价</b></td>')
            email_comment.append('<td align="center"><b>现涨跌幅</b></td>')
            email_comment.append('<td align="center"><b>总涨跌幅</b></td>')
            email_comment.append('<td align="center"><b>现市值</b></td>')
            email_comment.append('<td align="center"><b>盈亏额</b></td>')
            email_comment.append('<td align="center"><b>异动时间</b></td>')
            email_comment.append('</tr>')

            email_comment.append('<tr>')
            email_comment.append('<td align="center">'+str(1)+'</td>')
            email_comment.append('<td align="center">'+str(cbj) + '</td>')
            email_comment.append('<td align="center">' + str(shuliang) + '</td>')
            email_comment.append('<td align="center">' + str(now_jiage) +'</td>')
            email_comment.append('<td align="center">' + str(now_zdie) + '%</td>')
            email_comment.append('<td align="center">' + str(all_zdie) + '%</td>')
            email_comment.append('<td align="center">' + str(now_shizhi) + '元</td>')
            email_comment.append('<td align="center">' + str(ykui) + '元</td>')
            email_comment.append('<td align="center">' + str(riqi) +' '+str(sj) +'</td>')
            email_comment.append('</tr>')
            email_comment.append('</table>')
            email_comment.append('<p><font size="2" color="black">祝：股市天天红，日日发大财！</font></p>')
            email_comment.append('</html>')
            send_msg = '\n'.join(email_comment)
            send_Email(email_add[0], send_msg)

### 参考来源：https://zhuanlan.zhihu.com/p/410152581